In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn
import pandas
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
from scipy import misc
import os
import matplotlib.pyplot as plt
import csv
def read_csv_infos(file):

	idx = []
	labels = []
	sizesequences = []

	with open(file, 'r') as f:
		data = csv.reader(f, delimiter=' ')
		for d in data:
			j = d[0].split(',')
			idx.append(int(j[0]))
			labels.append(int(j[1]))
			sizesequences.append(int(j[2]))

	return idx, labels, sizesequences

In [0]:
def read_csv_skeleton(file, dim_joints):

	n = []
	n_samples = 0
	n_frames = 0
	with open(file, 'r') as f:
		data = csv.reader(f, delimiter=' ')
		for d in data:
			n_samples += 1
			n_frames=0
			for j in d[0].split(','):
				n.append(float(j))
				n_frames += 1
    # convert data in numpy array
	n = np.asarray(n)
	n = np.reshape(n, (n_samples, n_frames//(22*dim_joints), 22, dim_joints))
	return n

In [0]:
skeletons2=read_csv_skeleton("/content/drive/My Drive/data/skeletons_image_train.csv",2)
idx, labels, sequences=read_csv_infos("/content/drive/My Drive/data/infos_train.csv")

In [7]:
skeletons2.shape

(1960, 171, 22, 2)

In [0]:
skeletons3=read_csv_skeleton("/content/drive/My Drive/data/skeletons_world_train.csv",3)

In [9]:
skeletons3.shape

(1960, 171, 22, 3)

In [64]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout,Flatten, Dense, LeakyReLU

model = Sequential()

# Must define the input shape in the first layer of the neural network
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", 
          input_shape=(171,22,3), activation='relu'))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(256))
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(LeakyReLU())
model.add(Dense(29, activation='softmax'))

# Take a look at the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 171, 22, 32)       896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 171, 22, 64)       18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 85, 11, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 85, 11, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 85, 11, 128)       73856     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 83, 9, 256)        295168    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 27, 3, 256)       

In [0]:
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [0]:
labels=np.array(labels)

In [0]:

x=skeletons3
n=x.shape[0]
y=labels
selection=np.arange(n)
np.random.shuffle(selection)
subselection=selection[:int(n*0.6)]
x_train=x[subselection]
x_test=x[[i for i in np.arange(n) if i not in subselection]]
y_train=y[subselection]
y_test=y[[i for i in np.arange(n) if i not in subselection]]

In [68]:
model.fit(x, y,
         epochs=100,
          batch_size=64#,
          #validation_data=(x_test, y_test)
         )

Epoch 1/100
1960/1960 [==============================] - 3s 1ms/step - loss: 3.2412 - acc: 0.0679
Epoch 2/100
1960/1960 [==============================] - 2s 956us/step - loss: 2.9466 - acc: 0.1158
Epoch 3/100
1960/1960 [==============================] - 2s 965us/step - loss: 2.5601 - acc: 0.2071
Epoch 4/100
1960/1960 [==============================] - 2s 965us/step - loss: 2.1233 - acc: 0.3245
Epoch 5/100
1960/1960 [==============================] - 2s 976us/step - loss: 1.7707 - acc: 0.4281
Epoch 6/100
1960/1960 [==============================] - 2s 968us/step - loss: 1.5672 - acc: 0.4786
Epoch 7/100
1960/1960 [==============================] - 2s 969us/step - loss: 1.3976 - acc: 0.5505
Epoch 8/100
1960/1960 [==============================] - 2s 975us/step - loss: 1.2413 - acc: 0.5913
Epoch 9/100
1960/1960 [==============================] - 2s 970us/step - loss: 1.1501 - acc: 0.6214
Epoch 10/100
1960/1960 [==============================] - 2s 957us/step - loss: 1.1017 - acc: 0.6464
E

In [17]:
model.evaluate(x_test,y_test)

784/784 [==============================] - 0s 615us/step


[1.58874727025324, 0.7219387755102041]

In [0]:
evaluation_data = read_csv_skeleton("/content/drive/My Drive/data/skeletons_world_test.csv",3)

In [70]:
print(evaluation_data.shape)
print(skeletons3.shape)

(840, 162, 22, 3)
(1960, 171, 22, 3)


In [0]:
npad = ((0, 0), (0, 9), (0, 0), (0, 0))
eval_pad = np.pad(evaluation_data, pad_width=npad, mode='constant', constant_values=0)

In [0]:
pred = model.predict_classes(eval_pad)
results = [[i, pred[i]] for i in range(840)]
np.savetxt("results_v1.csv", results, delimiter=",", header='id,prediction')

In [0]:
# a = np.array([int(np.floor(i/30))+1 for i in range(840)])

In [99]:
pred

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  1,  9,  2,  2,  1,  1,
       10,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        8,  8,  8,  8,  8,  8,  2,  3,  3,  3,  1, 11,  3,  3,  3, 11,  5,
        3,  9,  3,  3,  3,  9, 27,  7,  1,  1,  1,  3,  7,  7,  3,  5,  7,
        3,  5,  5,  5,  5,  7,  2, 28,  4,  4,  4, 10, 20,  4,  9,  4,  4,
        4,  4,  8,  4,  4,  4,  4,  8,  4,  4,  4,  4,  4,  4,  4,  8,  8,
        3,  5,  5,  5,  6,  6,  6,  6,  5,  7,  5,  5,  5,  5, 17,  9,  5,
       17,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  4,  4,  6,  6,
        6,  6,  4,  6,  6,  6,  6,  7,  5,  1, 18,  6, 17,  6,  6,  6,  6,
        6,  6,  6,  6,  7,  7,  7,  8,  7,  7,  7,  7,  7,  7,  4,  6,  7,
        7,  3,  1,  1,  1,  9,  1,  7,  1,  1,  7,  7,  7,  7,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8, 10,  8,  2, 10,  8,  8,  8,
        8,  2,  8,  2, 10

In [100]:
0.65079*840

546.6636